# Processor Luncher Function

In [1]:
import nuclio

In [2]:
# nuclio: start-code

In [6]:
from mlrun import import_function
from mlrun import DataItem

In [22]:
def preprocessor(context,
                 df_artifact: DataItem,
                 preprocessor_fn: str = 'db://network-oeprations/nuclio-preprocessor',
                 aggregate_fn_url: str = 'hub://aggregate',
                 metrics_table: str = 'metrics',
                 features_table: str = 'features',
                 suffix: str = '', 
                 keys: str = None,
                 window: int = 3, 
                 center: bool = False, 
                 inplace: bool = False,
                 drop_na: bool = True):
    
    # Load base file and extract the aggregation parameters from it
    context.logger.info('Extracting aggregation data')
    df = df_artifact.as_df()
    selected_features = list(df.columns)
    aggregated_features = [feature.split('_')[:-1] for feature in selected_features if feature.endswith(suffix)]
    base_features = list(set([f[0] for f in aggregated_features]))
    aggregations = list(set([f[1] for f in aggregated_features]))
    
    # Set preprocessor nuclio function
    context.logger.info('Setting up streaming preprocessing function')
#     fn = import_function(preprocessor_fn)
#     fn.set_envs({'suffix': suffix,
#                  'keys': keys,
#                  'window': window,
#                  'center': center,
#                  'inplace': inplace,
#                  'drop_na': drop_na,
#                  'metrics': base_features,
#                  'metric_aggs': aggregations,
#                  'metrics_table': os.path.join(context.artifact_path, metrics_table),
#                  'features_table': os.path.join(context.artifact_path, features_table),
#                  'aggregate_fn_url': aggregate_fn_url})
#     fn.deploy(project=context.project)
    context.logger.info('Function sent for deployment')

## Test

In [23]:
from mlrun import code_to_function, mount_v3io, run_local

## Local

## Cluster

In [28]:
fn = code_to_function('preprocessor',
                      kind='job',
                      image='mlrun/ml-models',
                      handler='preprocessor')
fn.apply(mount_v3io())

In [31]:
fn.run(project='network-operations',
       artifact_path='/User/demo-network-operations/artifacts/',
       inputs={'df_artifact': '/User/demo-network-operations/artifacts/selected_features.parquet'})

[mlrun] 2020-06-28 10:09:47,036 starting run preprocessor-preprocessor uid=ef06421dd39c464fa3f21682a43d0bca  -> http://mlrun-api:8080


2020-06-28 10:10:07,062 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f91b49cdcc0>, 'Connection to mlrun-api timed out. (connect timeout=20)')': /api/func/default/preprocessor?versioned=True
2020-06-28 10:10:29,086 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f91b482aa90>, 'Connection to mlrun-api timed out. (connect timeout=20)')': /api/func/default/preprocessor?versioned=True
2020-06-28 10:10:33,253 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f91b48edb70>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /api/func/default/preprocessor?v

RunDBError: store function default/preprocessor